In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import pandas as pd


In [2]:
#Saves varaible with local dir
cwd = os.getcwd()
#Sets up webdriver to save files to local dir
options = webdriver.ChromeOptions() 
prefs = {"download.default_directory" : cwd}
#example: prefs = {"download.default_directory" : "C:\Tutorial\down"};
options.add_experimental_option("prefs",prefs)

In [3]:
#Creates driver object
driver = webdriver.Chrome(options=options)

#Load EDGAR website
driver.get(r'https://www.sec.gov/edgar/searchedgar/companysearch')
time.sleep(1)#Sleep to allow it to run


In [4]:
#Xpath to search box on edgar
xpath_search_box = r'//*[@id="edgar-company-person"]'

#Send the keystroke for requested 'ticker' followed by enter key to searchbox
ticker = 'NEE' #REMEMBER TO CHANGE THIS TO USE INPUT ARGUEMENT LATER!
driver.find_element("xpath", xpath_search_box).send_keys(ticker, Keys.ENTER)
time.sleep(1) #Sleep to allow it to run


In [6]:
#Now we need to click on "10-K (annual reports)..." to expand collapsible
xpath_10k_expand = r'//*[@id="filingsStart"]/div[2]/div[3]/h5'
driver.find_element("xpath", xpath_10k_expand).click()
time.sleep(1) #Sleep to allow it to run

#Now click on "View all 10-ks and 10-Qs"
xpath_10k_all= r'//*[@id="filingsStart"]/div[2]/div[3]/div/button[1]'
driver.find_element("xpath", xpath_10k_all).click()
time.sleep(1) #Sleep to allow it to run


In [7]:
#Enter "10-k" in search box and then end
#Send the keystroke for requested 'ticker' followed by enter key to searchbox
xpath_filings = r'//*[@id="searchbox"]'
driver.find_element("xpath", xpath_filings).send_keys("10-k", Keys.ENTER)
time.sleep(1) #Sleep to allow it to run

In [8]:
#Deletes any previously downloaded edgar files that might crash the script
try:
    os.remove("EDGAR Entity Landing Page.xlsx")
except OSError:
    pass


In [9]:

#Downloads  excel to get links to all filings
xpath_filings_excelbutton = r'//*[@id="filingsTable_wrapper"]/div[1]/button[3]'
driver.find_element("xpath", xpath_filings_excelbutton).click()
time.sleep(2) #Sleep to allow it to run


In [10]:

#Closes driver
driver.close()

#renames file name to ticker_edgar_filings.xlsx, deletes if exists
temp_file_name = f"{ticker}_edgar_filings.xlsx"
try:
    os.rename("EDGAR Entity Landing Page.xlsx", temp_file_name)    
except FileExistsError:
    os.remove(temp_file_name)
    os.rename("EDGAR Entity Landing Page.xlsx", temp_file_name)



In [11]:
#import to pandas df
filings_raw_df = pd.read_excel(temp_file_name)

#Drop unneeded columns and rename last to url
cleaned_df = filings_raw_df.drop(columns=["EDGAR Entity Landing Page", "Unnamed: 1", "Unnamed: 2","Unnamed: 3"])

#rename column, drop first 2 rows and missing values, reset index
cleaned_df = cleaned_df.rename(columns=cleaned_df.iloc[1])
cleaned_df = cleaned_df.drop([0,1])
cleaned_df = cleaned_df.dropna(axis = 0) #
cleaned_df.reset_index(drop=True, inplace=True)

#Outputs list of all rls
list_of_urls = cleaned_df["Filings URL"].values.tolist()

display(list_of_urls)

#Delete excel file after done
try:
    os.remove(temp_file_name)    
except FileNotFoundError:
    pass

['https://www.sec.gov/Archives/edgar/data/753308/000075330823000019/0000753308-23-000019-index.htm',
 'https://www.sec.gov/Archives/edgar/data/753308/000075330822000014/0000753308-22-000014-index.htm',
 'https://www.sec.gov/Archives/edgar/data/753308/000075330821000014/0000753308-21-000014-index.htm',
 'https://www.sec.gov/Archives/edgar/data/753308/000075330820000021/0000753308-20-000021-index.htm',
 'https://www.sec.gov/Archives/edgar/data/753308/000075330819000039/0000753308-19-000039-index.htm',
 'https://www.sec.gov/Archives/edgar/data/753308/000075330818000013/0000753308-18-000013-index.htm',
 'https://www.sec.gov/Archives/edgar/data/753308/000075330817000060/0000753308-17-000060-index.htm',
 'https://www.sec.gov/Archives/edgar/data/753308/000075330816000337/0000753308-16-000337-index.htm',
 'https://www.sec.gov/Archives/edgar/data/753308/000075330815000079/0000753308-15-000079-index.htm']